In [1]:
#Exportamos delayed
from dask import delayed
#Para leer el archivo csv como dataframe
from dask import dataframe
trips_df = dataframe.read_csv("/data/trips.csv")
trips_df.head()

,car_type,fare_amount,passenger_count,taxi_id,tip_amount,tpep_dropoff_datetime,tpep_pickup_datetime,trip_distance
0,A,22.0,1,1,4.60,2015-01-03 01:37:02,2015-01-03 01:17:32,6.90
1,A,9.0,1,1,0.00,2015-01-05 23:35:02,2015-01-05 23:25:15,1.81
2,A,7.5,1,1,1.00,2015-01-06 15:22:12,2015-01-06 15:11:45,0.96
3,A,8.5,1,1,1.00,2015-01-08 08:31:23,2015-01-08 08:22:12,1.90
4,A,7.5,1,1,1.66,2015-01-08 12:35:54,2015-01-08 12:26:26,1.00


In [2]:
# Truco para convertir a datetime...
trips_df.tpep_pickup_datetime = trips_df.tpep_pickup_datetime.astype('M8[us]')
trips_df.tpep_dropoff_datetime = trips_df.tpep_dropoff_datetime.astype('M8[us]')
trips_df.tpep_dropoff_datetime.head()

0   2015-01-03 01:37:02
1   2015-01-05 23:35:02
2   2015-01-06 15:22:12
3   2015-01-08 08:31:23
4   2015-01-08 12:35:54
Name: tpep_dropoff_datetime, dtype: datetime64[ns]

In [3]:

trips_df["trip_duration"] = trips_df.tpep_dropoff_datetime - trips_df.tpep_pickup_datetime
trips_df.trip_duration.map(lambda d: d.total_seconds()).head()

0    1170.0
1     587.0
2     627.0
3     551.0
4     568.0
Name: trip_duration, dtype: float64

In [4]:

trips_df.head()

,car_type,fare_amount,passenger_count,taxi_id,tip_amount,tpep_dropoff_datetime,tpep_pickup_datetime,trip_distance,trip_duration
0,A,22.0,1,1,4.60,2015-01-03 01:37:02,2015-01-03 01:17:32,6.90,00:19:30
1,A,9.0,1,1,0.00,2015-01-05 23:35:02,2015-01-05 23:25:15,1.81,00:09:47
2,A,7.5,1,1,1.00,2015-01-06 15:22:12,2015-01-06 15:11:45,0.96,00:10:27
3,A,8.5,1,1,1.00,2015-01-08 08:31:23,2015-01-08 08:22:12,1.90,00:09:11
4,A,7.5,1,1,1.66,2015-01-08 12:35:54,2015-01-08 12:26:26,1.00,00:09:28


In [ ]:
Creamos la proporcion de propina

In [18]:

trips_df["prop"] = trips_df.tip_amount / trips_df.fare_amount
trips_df["prop"] =trips_df["prop"] *100

trips_df.head()

,car_type,fare_amount,passenger_count,taxi_id,tip_amount,tpep_dropoff_datetime,tpep_pickup_datetime,trip_distance,trip_duration,prop
0,A,22.0,1,1,4.60,2015-01-03 01:37:02,2015-01-03 01:17:32,6.90,00:19:30,20.909091
1,A,9.0,1,1,0.00,2015-01-05 23:35:02,2015-01-05 23:25:15,1.81,00:09:47,0.000000
2,A,7.5,1,1,1.00,2015-01-06 15:22:12,2015-01-06 15:11:45,0.96,00:10:27,13.333333
3,A,8.5,1,1,1.00,2015-01-08 08:31:23,2015-01-08 08:22:12,1.90,00:09:11,11.764706
4,A,7.5,1,1,1.66,2015-01-08 12:35:54,2015-01-08 12:26:26,1.00,00:09:28,22.133333


In [ ]:
Primero vamos a ver el python secuencial usando sklearn

In [25]:

import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [27]:
df = pd.read_csv("/data/trips.csv")
df = pd.DataFrame(df)

df['prop'] =  df.tip_amount / df.fare_amount
df['prop'] = df['prop']*100

df = df[['prop','trip_distance',"car_type"]]
df = df.dropna(axis=0, how='any')
label_encoder = LabelEncoder()
df["car_type"] = label_encoder.fit_transform(df["car_type"])
df[0:10]

,prop,trip_distance,car_type
0,20.909091,6.90,0
1,0.000000,1.81,0
2,13.333333,0.96,0
3,11.764706,1.90,0
4,22.133333,1.00,0
5,15.789474,1.71,0
6,18.750000,1.27,0
7,21.333333,1.55,0
8,26.000000,0.54,0
9,57.692308,15.38,0


In [28]:
y = df.prop
#Separo mis datos en entrenamiento y prueba, utilizaré el 20%
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2)
#Tamaño de mi conjunto de prueba
X_train.shape, y_train.shape

((7356, 3), (7356,))

In [29]:
#Tamaño de mi conjunto de prueba
X_test.shape, y_test.shape

((1839, 3), (1839,))

In [32]:
#Ajustamos un modelo de regresión lineal
lm = linear_model.LinearRegression(fit_intercept=False)
model = lm.fit(X_train, y_train)
predictions = lm.predict(X_test)



Ahora correremos una busqueda en grid para encontrar los mejores parámetros

In [36]:
%%time
import numpy as np
from sklearn.grid_search import GridSearchCV
model = linear_model.LinearRegression()
parameters = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False], 'n_jobs':[50,100,500]}
grid = GridSearchCV(model,parameters, cv=10)
grid.fit(X_train, y_train)
"r2 / variance : ", grid.best_score_
grid.best_score_
"Residual sum of squares: %.2f"% np.mean((grid.predict(X_test) - y_test) ** 2)

CPU times: user 950 ms, sys: 0 ns, total: 950 ms
Wall time: 1 s


In [39]:
"Mejores parámetros  : ", grid.best_params_

('Mejores parámetros  : ',
 {'copy_X': True, 'fit_intercept': True, 'n_jobs': 50, 'normalize': True})

Ahora vamos a hacer lo mismo pero con dask ML

In [43]:
%%time
from dask.distributed import Client
client = Client()  # start a local Dask client

import dask_ml.joblib
from sklearn.externals.joblib import parallel_backend
with parallel_backend('dask'):
    from sklearn.grid_search import GridSearchCV
    model = linear_model.LinearRegression()
    parameters = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False], 'n_jobs':[50,100,500]}
    grid_dask = GridSearchCV(model,parameters, cv=10)
    grid_dask.fit(X_train, y_train)
    "Residual sum of squares: %.2f"% np.mean((grid_dask.predict(X_test) - y_test) ** 2)
    grid_dask.best_params_

CPU times: user 2.59 s, sys: 2.08 s, total: 4.67 s
Wall time: 7.65 s


In [44]:
"Mejores parámetros con Dask : ",grid_dask.best_params_

{'copy_X': True, 'fit_intercept': True, 'n_jobs': 50, 'normalize': True}

Finalmente usaremos la función delayed

In [42]:

def paralelo(A):
    from sklearn.grid_search import GridSearchCV
    model = linear_model.LinearRegression()
    parameters = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False], 'n_jobs':[50,100,500]}
    grid = GridSearchCV(model,parameters, cv=3)
    grid.fit(X_train, y_train)
    return grid.best_params_

delayed_grid = delayed(grid)
print("Paralelo:")
%time paralelo(df)

Paralelo:
CPU times: user 380 ms, sys: 0 ns, total: 380 ms
Wall time: 419 ms


{'copy_X': True, 'fit_intercept': True, 'n_jobs': 50, 'normalize': True}

Una vez que corrimos los tres modelos podemos concluir que el tiempo de la función delayed fue el mejor de los 3, seguido por el sklearn y finalmente daskML